# Milling Project 

## Section 1: Introduction 

<img src="./intro_mill_geometry.png" width=400 />

### Section 1.1: Problem Description 

Numerical modeling of the milling of paper pulp using a pressure-driven rotating disk mill.

### Section 2.1: Questions 

1. water vs. pulp (particle size distribution and particle surface tension/hardness): change in viscosity - local changes in strain and shear stress; 
2. effect to temperature and pre-heating; 
3. surface roughness: measured values, observed effects, parameter setting for turbulent viscosity; 

### Section 3.1: Remarks

1. in early geometry models outlet too close to main flow domain;
2. non-linear laminar viscosity and turbulent are expected to play complementary roles. We expect that both will need to be modeled.  
3. we suggest to place sufficiently anonymous geometry in public domain; 
4. we suggest to revert to OpenFoam to automate workflow and to make large scale simulations (in case applicable) possible; 

### Section 4.1: State of the Art 

What literature/state-of-the-art do we base our work on? 

### Section 5.1: Research Question 

1. mesh independent results for shear stress in the airgap (otherwise results numerics induced and thus hard to compare).  
2. note: lab data should be used as boundary conditions in the mathematical model. 

### Section 6.1: Notes on these Notes (aka External Memorry) 

<b>To do: 0-directory and system/graphs</b>

1. define case set-up in case 0-directory;
2. define post-processing in the system/graphs file;
3. add mass flow rate at the outlet;
4. refine gnuplot functionality to view the shear stress in the airgap;   

<b>Goals: Non-Isothermal RANS Flow Simulations with Water as Newtonian Fluid</b>

1. obtain mesh-independent results for the shear stress in the airgap. Else impossible to access whether results are due to the RANS turbulence closure model (standard k-epsilon model or otherwise) or the numerics of the discretization;   
2. perform parameter studies to access how flow quantities of interest (the shear stress in the air gap, the outlet mass flow rate) depend on operating conditions (the angular velocity of the rotor, the surface roughess of the rotor, temperature of the fluid, viscosity of the fluid (artifically modified), the thickness of the airgap) and the geometry (outer boundaries, stent?) of the flow domain. Ask for guesses by experiments; 

<b>To do: Non-Isothermal RANS Flow Simulations with Water as Newtonian Fluid</b>
1. rotating disk - option-1 (easiest? - do we require alternatives?): implement rotating disk as in the [pipeCyclic tutorial case](https://develop.openfoam.com/Development/openfoam/-/tree/master/tutorials/incompressible/simpleFoam/pipeCyclic) using a codedFixedValue for U (assuming no-slip between disk and liquid. Compare with lid driven cavity);
2. rotating disk - option-2: using Multiple Referende Frame: [details](https://www.openfoam.com/documentation/guides/v2206/doc/verification-validation-rotating-cylinders-2d.html) and [rotatingCylinders tutorial case](https://develop.openfoam.com/Development/openfoam/-/tree/master/tutorials/incompressible/simpleFoam/rotatingCylinders) 
3. rotating disk - option-3: [rotorDisk](https://www.openfoam.com/documentation/guides/v2206/doc/guide-fvoptions-sources-rotor-disk.html) as alternative for setting rotating rotor - does rotorDisk allow to specify a laminar surface roughness;
4. surface roughness of disk (1/3): laminar flow settings:  dixit <i>Laminar flow is independent of the pipe roughness since the flow is stratified and covers the roughness. It then behaves like a flow along a smooth wall.</i> (see top page 8 [Internship report](https://static.fossee.in/fossee/FOSSEE-Summer-Fellowship2022/OpenFOAM/Tinto_Report_Friction_Factor_LM.pdf)) Google Gemini gives same answer. 
5. surface roughness of disk (2/3): RANS turbulent flow settings: wall function for turbulent viscosity for rough walls. See [nutURoughWallFunction](https://www.openfoam.com/documentation/guides/v2206/api/classFoam_1_1nutURoughWallFunctionFvPatchScalarField.html#details). How sensitve is the model to model parameters?
6. surface roughness of disk (3/3): Should (D)DES or Wall-Modeled LES serve as alternative to RANS?  
7. post-processing for the shear in case of solving for the shear stress (S), add visualization of the shear stress;

## Section 2: Governing Fluid Equations 

### Section 1.2: Newtonian Fluid 

Describe flow equations for Newtonian fluid 
1. stationary incompressible (low Mach) variable-density ($\partial \rho / \partial x_i \neq 0$) flow using the conservation of mass, momentum and energy (using internal energy as variable and conversion from energy to temperature) for Reynolds-averaged non-isothermal turbulent flow of water (temperature dependence of thermal conductivity, heat capacity and density) and non-Newtonian fluid
2. equation of state for the fluid - use of thermodynamics;
3. include friction as volumetric source term in the energy equation; 
4. volumetric turbulence using RANS closure models;
5. wall turbulence modeling of walls using wall functions for velocity and temperature (turbulent Prandtl number set to 0.85 in simulation);
6. modeling to rotating surface by prescribing the velocity on the rotating surface; 
7. modeling of rotor surface roughness using the wall function for the turbulent viscosity. How to set model parameters? How sensitive is the model to model parameters? 
8. inlet and outlet conditions including inlet turbulent intensity (5 procent based on hydraulic diameter?), turbulent dissipation rate
9. material properties: molecular viscosity of water, kinematic viscosity of water, thermal conductivity of water;  
10. describe dimensionless number: Reynolds number and [Strouhal number](https://en.wikipedia.org/wiki/Strouhal_number); 
11. finite volume discretization and segregated (SIMPLE-based) solution approach using rhoSimpleFoam. See [description of rhoSimpleFoam](https://openfoamwiki.net/index.php/RhoSimpleFoam) (requires more text to explain the viscous stress tensor);  

### Section 2.2: Modeling Scalar Viscosity for Shear Thinning Fluid 

Describe flow equations for non-Newtonian fluid with shear-thinning assuming that stress($\tau$)-strain($\epsilon$) relation $\tau = \mu \sigma$ still holds 
1. allow the molecular viscosity to varry according to a power-law or variant thereof;
3. describe [Deborah number](https://en.wikipedia.org/wiki/Deborah_number) and the [Weisenberg number](https://en.wikipedia.org/wiki/Weissenberg_number) 

### Section 3.2: Modeling Stress Tensor for Shear Thinning Fluid
2. describe flow equations for non-Newtonian fluid with shear-thinning allowing the symmetric stress tensor (3(6) componentsw in 2D(3D)) to varry according to a Maxwell or Oldroyd-B model;   

### Section 4.2: Open Ends 
1. extension to non-stationary flow required for small gaps?
2. conjugate heat transfer (fluid-solid modeling) to take thermal stiffness of the casing of the fluid domain into account?  
3. use of thin film approximation and finite area meshing (faMesh) to describe flow behavior in thin gap between rotor and stator. Requires thin film equations for shear thinning fluid;
4. model out of plane motion of the rotor by taking the forces of the fluid into account;
5. shape optimization for stator shape to minimize the pressure drop (and thus the energy consumed);
6. topology optimization for obstacle in the fluid domain steering flow in desired direction; 

## Section 3: Pre-Processing 

### Section 1.3: 2D or 3D Geometry Definition using FMS Files 

Geometry defined by FMS-file. In FMS file, one patch for each node on boundary of the front patch. Each patch consists of two triangles.  

### Section 2.3: Mesh Generation using cartesian(2D)Mesh and extrudeMesh 

1. 2D mesh generation using cartesian2DMesh with system/meshDict (cell size, local refinement regions, boundary layers and more) as input;
2. 3D mesh generation using cartesianMesh with system/meshDict
7. mesh extrusion from 2D to wedge using extrudeMesh with system/extrudeMeshDict as input. After the extrusion, the patch Axis contains no faces. The patch Axis can be thus be excluded from further analysis.

### Section 3.3: Mesh Quality Statistics using checkMesh 

1. check quality of the volumetric mesh using checkMesh;
2. check quality of the wall boundary mesh by computing y+ in post-processing phase; 

<img src="./checkMesh_extrudeMesh_cartesian2Dmesh.png" width=400 />

### Section 4.3: Automated Definition of Airgap Thickness using M4 Preprocessor 

Use [M4 Preprocessor](https://www.gnu.org/software/m4/manual/html_node/index.html) to change. See <i>tutorials/multiphase/interIsoFoam/sloshingTank2D/system/</i> for an example. 
See other tutorials for more examples. 

### Section 5.3: Known Limitations of Current Meshing Approach 

Current approach fails in case that airgap is too thin. Requires other approach to local refinement or the possibility to smoothen the interface between refinement zones and their exterior.  

### Section 6.3: Conversion to AnsysFluent using foamMeshToFluent 

Convert meshes for use with AnsysFluent using foamMeshToFluent  

## Section 4: Flow Simulations 

### Section 1.4: Relevant OpenFoam Information 

1. [case structure](https://www.openfoam.com/documentation/guides/latest/doc/openfoam-guide-case-structure.html)
2. [boundary conditions](https://www.openfoam.com/documentation/guides/v2206/doc/openfoam-guide-boundary-conditions.html)

### Section 2.4: Laminar Flow using simpleFoam

Solve pressure-velocity using SIMPLE algorithm implemented in [simpleFoam](https://www.openfoam.com/documentation/guides/v2112/man/simpleFoam.html). 

### Section 3.4: Turbulent Flow using simpleFoam 
`
Solve pressure-velocity and turbulent quantities using SIMPLE algorithm.

### Section 4.4: Non-Newtonian Turbulent Flow using simpleFoam

Solve pressure-velocity and turbulent quantities for non-Newtonian flow using SIMPLE algorithm.

Laminar flow models: see [generalizedNewtonianViscosityModels](https://www.openfoam.com/documentation/guides/v2206/api/namespaceFoam_1_1laminarModels_1_1generalizedNewtonianViscosityModels.html) 

Turbulent flow: [laminar transport](https://www.openfoam.com/documentation/guides/v2206/api/group__grpLaminar.html)

### Section 5.4: Non-Isothermal Flow using rhoSimpleFoam

Solve pressure-velocity-temperature and turbulent quantities for (non-)Newtonian flow using SIMPLE algorithm implemented in [rhoSimpleFoam](https://www.openfoam.com/documentation/guides/v2112/man/rhoSimpleFoam.html).

## Section 5: Post-Processing 

### Section 1.5: Plots Generated During the Simulations  

Line plots for shear stress in the airgap and mass flow rate at the outlet.  

### Section 2.5: Generating Additional Fields using postProcess 

Generating e.g. the Reynolds stress tensor or the residual field using postProcess.  

### Section 3.5: Viewing Fields using Paraview 

View fields using Paraview. 

### Section 4.5: Viewing Runtime and Convergence Statistics using foamLog

View residual convergence history and number of iterations for the linear solver. 

## Section 6: OpenFoam Cases 

### Section 1.6: easyMillCartesian2DMesh

<b>Objective</b>
Demonstrate geometry definition and mesh generation. 

<b>Known Limitations</b>
Air gap large enough to allow for a uniform mesh (avoid local refinement zones.)

<b>Partial Solutions</b>
Replace rectangular refinement regions by wedge refinement region. 

<img src="./mill_cartesian2d_mesh_zoom1.png" width=400 />
<img src="./large_residual_mesh_interface.png" width=400 />

### Section 2.6: easyMill3DMesh (requires care)

<b>Objective</b>
Verify whether cartesianMesh (3D variant), tetMesh (tetrahedral meshes) or pMesh (polyhedral meshing) offer distinct advantages over cartesian2DMesh and extrudeMesh. Requires further documentation. 

<b>Questions</b>

1. <div class="alert alert-block alert-warning">  project requires more documentation on FMS files (visualization) </div>
2. <div class="alert alert-block alert-warning">  cartesian2DMesh and cartesianMesh very different output to screen. Unclear why? </div>
3. <div class="alert alert-block alert-warning"> can additional smoothing steps in cartesian2DMesh be enforced? </div>
4. <div class="alert alert-block alert-warning"> can 2D meshing in cartesianMesh be enforced? </div> 

### Section 3.6: easyMillLaminarSimpleFoam 

<b>Objective</b>
Demonstrate segragated solution of pressure-velocity coupling and post-processing capabilities.  

<b>Known Limitations</b>
Works for low Reynolds numbers only. We set a low inlet velocity ($0.1$ m/s) and increased the kinematic viscosity of the fluid. 

### Section 4.6: easyMillSimpleFoam

<b>Objective</b>
Demonstrate segragated solution of pressure-velocity-turbulence coupling. 

<b>Known Limitations</b> 
No pressure inlet velocity. No temperature information. 

### Section 5.6: easyMillrhoMillSimpleFoam

<b>Objective</b>
Demonstrate segragated solution of pressure-velocity-temperature-turbulence coupling. 

<b>Known Limitations</b> 
No transient (unsteady) effects.

### Section 6.6: Extensions 

## Section 7: References 

1. references in <i>Project Plan for CFD Model</i>;
2. [Stafford-2025](https://journals.aps.org/prfluids/pdf/10.1103/PhysRevFluids.10.024805) in Physical Review Physics 2025: thin film over spinning disk; 
3. [Kumar-2016](https://www.researchgate.net/publication/305737251_Rheology_of_Microfibrillated_Cellulose_Suspensions_in_Pressure-Driven_Flow) in Applied Rheology 2016: shear thinning expressed as a power law, a Casson law or a Herschel-Buckley law;  

## Sandbox 

In [1]:
    # rotating disk implementation; 
    inlet
    {
        type            codedFixedValue;
        value           uniform (1 0 0);
        name            swirl;
        code            #{
            // Uncomment for testing on non-windows systems [fragile]
            // IOobject::scopeSeparator = '_';

            const vector axis(1, 0, 0);

            vectorField v(2.0*this->patch().Cf() ^ axis);
            v.replace(vector::X, 1.0);
            operator==(v);
        #};
    }

LoadError: UndefVarError: `inlet` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [2]:
# mass flow rate the outlet  
functions
{
    surfaceFieldValue1
    {
        type            surfaceFieldValue;
        libs            ("libfieldFunctionObjects.so");
        enabled         yes;
        writeControl    writeTime;
        log             yes;
        writeFields     no;
        regionType      patch;
        name            outlet;
        operation       weightedAverage;
        weightField     phi;
        fields
        (
            H2O
            T
        );
    }
}

LoadError: UndefVarError: `functions` not defined in `Main`
Suggestion: check for spelling errors or missing imports.